In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/language-detection/Language Detection.csv


In [39]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense,LSTM,Embedding,Dropout,Bidirectional
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [6]:
df=pd.read_csv('/kaggle/input/language-detection/Language Detection.csv')
x_train, x_temp, y_train, y_temp = train_test_split(df['Text'], df['Language'], test_size=0.4, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [10]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(6202,)
(6202,)
(2067,)
(2067,)


In [12]:
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
vocab_size=10000

tokenizer=Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)
train_x = tokenizer.texts_to_matrix(x_train, mode='tfidf')
valid_x = tokenizer.texts_to_matrix(x_val, mode='tfidf')

encoder=LabelBinarizer()
encoder.fit(y_train)
train_y = encoder.transform(y_train)
valid_y = encoder.transform(y_val)

In [14]:
print(train_x.shape)
print(train_y.shape)
print(valid_x.shape)
print(valid_y.shape)

(6202, 10000)
(6202, 17)
(2067, 10000)
(2067, 17)


In [43]:
num_classes=17
embedding_dim=25
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=train_x.shape[1]))
model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
model.add(Dropout(0.25))

model.add(Bidirectional(LSTM(units=32)))
model.add(Dropout(0.25))

model.add(Dense(units=num_classes, activation='softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 10000, 25)         250000    
                                                                 
 bidirectional_4 (Bidirecti  (None, 10000, 128)        46080     
 onal)                                                           
                                                                 
 dropout_10 (Dropout)        (None, 10000, 128)        0         
                                                                 
 bidirectional_5 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 17)               

In [44]:
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
optimizer = RMSprop(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', #categorical cross...obtains the loss for multiple classes (i.e. categories entertainment, politcs...). Need to use with Softmax act function
              optimizer=optimizer, #adam is an optimization function to minimize the y' value in the loss function. A typical optimization function used in text classification.
              metrics=['accuracy']) #in the absence of confusion matrix, we can just use accuracy as metric.

history = model.fit(train_x, train_y,    #fitting the model with the training set (x-train), y-train(training set data's tags encoded) The annotated dataset.
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(valid_x,valid_y)
                    )

Epoch 1/10
194/194 [==============================] - 219s 1s/step - loss: 2.8084 - accuracy: 0.1250 - val_loss: 2.7610 - val_accuracy: 0.1413
Epoch 2/10
194/194 [==============================] - 208s 1s/step - loss: 2.7550 - accuracy: 0.1287 - val_loss: 2.7228 - val_accuracy: 0.1413
Epoch 3/10
194/194 [==============================] - 208s 1s/step - loss: 2.7462 - accuracy: 0.1261 - val_loss: 2.7146 - val_accuracy: 0.1413
Epoch 4/10
194/194 [==============================] - 208s 1s/step - loss: 2.7399 - accuracy: 0.1290 - val_loss: 2.7074 - val_accuracy: 0.1413
Epoch 5/10
194/194 [==============================] - 207s 1s/step - loss: 2.7371 - accuracy: 0.1287 - val_loss: 2.7026 - val_accuracy: 0.1413
Epoch 6/10
194/194 [==============================] - 208s 1s/step - loss: 2.7244 - accuracy: 0.1283 - val_loss: 2.6813 - val_accuracy: 0.1413
Epoch 7/10
194/194 [==============================] - 208s 1s/step - loss: 2.6816 - accuracy: 0.1258 - val_loss: 2.6095 - val_accuracy: 0.1413

In [27]:
text_labels=encoder.classes_
text_labels

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype='<U10')

In [45]:
for i in range(0, 100):  # Range can be adjusted based on the size of your test set
    # Assuming you have tokenized your sequences
    sequence = tokenizer.texts_to_sequences([x_test.iloc[i]])
    
    # If you used texts_to_matrix with mode='tfidf'
    input_for_prediction = tokenizer.texts_to_matrix([x_test.iloc[i]], mode='tfidf')
    
    # Make the prediction
    prediction = model.predict(input_for_prediction)
    
    # If it's multilabel classification, you might want to use a threshold
    threshold = 0.5
    predicted_labels = [text_labels[j] for j in range(num_classes) if prediction[0][j] > threshold]

    # Find the class with the maximum probability
    max_prob_class = text_labels[np.argmax(prediction[0])]

    # Print the true labels (ground truth)
    true_labels = y_test.iloc[i]
    print('Actual labels:', true_labels)
    
    # Print the predicted labels
    print('Predicted labels:', predicted_labels)
    
    # Print the class with the maximum probability
    print('Max Probability Class:', max_prob_class)
    
    # Print the raw prediction probabilities
    print('Prediction:', prediction)
    
    # Add a line break for better readability
    print('\n')


1/1 [==============================] - 2s 2s/step
Actual labels: French
Predicted labels: []
Max Probability Class: English
Prediction: [[0.01394256 0.03381861 0.03393769 0.2820863  0.18620247 0.01435912
  0.00965643 0.00268559 0.08325002 0.00551547 0.01038466 0.08519481
  0.02519833 0.13840945 0.05357739 0.00979183 0.01198929]]


1/1 [==============================] - 0s 424ms/step
Actual labels: English
Predicted labels: []
Max Probability Class: English
Prediction: [[0.01346353 0.03288299 0.03440708 0.285675   0.18136655 0.01443034
  0.00954793 0.00273919 0.0875532  0.00541221 0.00996743 0.08739632
  0.02604357 0.13339417 0.05384436 0.00955756 0.01231865]]


1/1 [==============================] - 0s 423ms/step
Actual labels: Russian
Predicted labels: []
Max Probability Class: Malayalam
Prediction: [[0.08116863 0.03562834 0.05191658 0.04970074 0.04068553 0.0665044
  0.05147141 0.01515236 0.03807509 0.09029626 0.09398334 0.03860547
  0.08831709 0.04984373 0.06293754 0.08323599 0.06247